In [1]:
import spacy
import pandas as pd
import numpy as np
import time
from statistics import mean 
from multiprocessing import Pool
import pickle
from spacy.tokens import DocBin

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

nlp = spacy.load("pt_core_news_sm")

In [2]:
df = pd.read_csv("MERCADO_FINAL.csv")
port = pd.read_csv("estaticos_portfolio1.csv")
df_port = df[df['id'].isin(port['id'])].reset_index(drop=True)

In [3]:
len(df)

462298

In [4]:
df

,id,0
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,sociedade empresaria limitada entidades empres...
1,6178f41ade1365e44bc2c46654c2c8c0eaae27dcb476c4...,empresario individual outros servicos alojamen...
2,4a7e5069a397f12fdd7fd57111d6dc5d3ba558958efc02...,empresario individual outros transporte armaze...
3,3348900fe63216a439d2e5238c79ddd46ede454df7b9d8...,empresario individual outros servicos diversos...
4,1f9bcabc9d3173c1fe769899e4fac14b053037b953a1e4...,empresario individual outros servicos profissi...
...,...,...
462293,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,empresario individual outros comercio reparaca...
462294,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,empresario individual outros servicos administ...
462295,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,associacao privada entidades fins lucrativos s...
462296,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,sociedade empresaria limitada entidades empres...


In [6]:
df_dupl = df.drop_duplicates(subset='0', keep="last").reset_index(drop=True)
df_port_dupl = df_port.drop_duplicates(subset='0', keep="last").reset_index(drop=True)
dici_dupl = df['0'].value_counts().to_dict()
dici_port = df_port['0'].value_counts().to_dict()

In [7]:
def tokeniza(df):
    df["Docs"] = df["0"].apply(lambda x: nlp(x)).to_list()
    return df

In [8]:
def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.terminate()
    pool.join()
    return df
#     lista_final=[]
#     for i in lista:
#         lista_final.append(i)

#     return lista_final

In [9]:
start_time = time.time()
df_dupl = parallelize_dataframe(df_dupl,tokeniza)
print("--- %s seconds ---" % (time.time() - start_time))

--- 46.66700553894043 seconds ---


In [10]:
start_time = time.time()
df_port_dupl = parallelize_dataframe(df_port_dupl,tokeniza)
print("--- %s seconds ---" % (time.time() - start_time))

--- 15.070544958114624 seconds ---


In [11]:
def compara(df):
    df_vazio=pd.DataFrame(columns=["id","similarity"])
    #itera mercado
    for i in range(len(df)):
        doc_mercado = df["Docs"].iloc[i]
        similarity_sum=0
        #iteraportfolio
        for j in range(len(df_port_dupl)):
            doc_portfolio = df_port_dupl["Docs"][j]
            similarity_sum+=(doc_mercado.similarity(doc_portfolio)*dici_port[df_port_dupl["0"][j]])
            semelhanca_media = similarity_sum/len(df_port)
        df_vazio = df_vazio.append({'id':df['id'].iloc[i],'similarity':semelhanca_media},ignore_index=True)
    #df["Similarity"]=df.apply(lambda row: (doc_mercado.similarity(row['Docs']) * dici_port[row["0"]]),axis=1)
    return df_vazio

In [12]:
def parallelize_compare(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split)).reset_index(drop=True)
    pool.close()
    pool.terminate()
    pool.join()
    return df

In [13]:
%%capture
start_time = time.time()
#itera mercado
df_vazio = parallelize_compare(df_dupl,compara)
tempo_total = (time.time() - start_time)
print("--- %s seconds ---" % (time.time() - start_time))

In [14]:
print("--- %s seconds ---" ,tempo_total)

--- %s seconds --- 2078.953675508499


In [15]:
df_vazio

,id,similarity
0,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,0.839776
1,764bb8648b96fd6459fa2f2237c22f3d6b8c129a814e33...,0.892028
2,a81c8e1d49c50f9ee7e5e7b25cd2223eba13ffcff402fe...,0.846356
3,128ad43b6d43ab63679620906fc4bcb733f7561513967d...,0.878410
4,a820394924e9d954f40431b995072bd5c4b8b7b1239d59...,0.815033
...,...,...
6827,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,0.924140
6828,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,0.868174
6829,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,0.857196
6830,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,0.922097


In [24]:
df_final = df_dupl.merge(df_vazio,on="id")[["id","0","similarity"]]

In [25]:
df_final

,id,0,similarity
0,6c45aabb849a760a3a7107c7a61262642f683feb8912a4...,cooperativa entidades empresariais bens consum...,0.839776
1,764bb8648b96fd6459fa2f2237c22f3d6b8c129a814e33...,consorcio sociedades entidades empresariais in...,0.892028
2,a81c8e1d49c50f9ee7e5e7b25cd2223eba13ffcff402fe...,sociedade empresaria limitada entidades empres...,0.846356
3,128ad43b6d43ab63679620906fc4bcb733f7561513967d...,orgao publico poder executivo municipal admini...,0.878410
4,a820394924e9d954f40431b995072bd5c4b8b7b1239d59...,empresa individual responsabilidade limitada n...,0.815033
...,...,...,...
6827,469ee5fc984d0bb60aa2f40cda4d003d7c9b963d149ed3...,empresario individual outros comercio reparaca...,0.924140
6828,656f496e7e2fad708f0022d8ad97d2cb9344e839a5f3cb...,empresario individual outros servicos administ...,0.868174
6829,9b933a5cf2149d1338f00fb459d34a16b9b0f2e8d3e8e1...,associacao privada entidades fins lucrativos s...,0.857196
6830,84db0f17467cf4f02439e01ade57150b6ac59d9dae53ed...,sociedade empresaria limitada entidades empres...,0.922097


In [34]:
df_final_mesmo = df.merge(df_final[["0","similarity"]],on="0")[["id","similarity"]]

In [35]:
df_final_mesmo

,id,similarity
0,a6984c3ae395090e3bee8ad63c3758b110de096d5d8195...,0.874088
1,5b15f115857339af828935c36ab535758541757162af4c...,0.874088
2,a1c576d8d8cc61e8e367d0b0bed5392bff26644ca586ed...,0.874088
3,1fce2cf54621fe7f5200cc7a72f2404a4920334a536ba9...,0.874088
4,b8ca26567452bd72f3da6bbc9c29b2ab587de55b751219...,0.874088
...,...,...
462293,db5a6c9b40549e5e990e3eb48181bc9bc35fe8821d7122...,0.858811
462294,2aa6f9b665ef82d5aea7b58e337aa938e6b69d7d0ed54c...,0.730174
462295,b95e2d03766d23165df88e75c0380c2df1a022b567aff2...,0.729299
462296,bc03cf7c46a7df96a3e76a15386807c3d9fdeab6e46f71...,0.860076


In [38]:
# Dropando portfolio do dataset mercado
df_final_mesmo = df_final_mesmo[~df_final_mesmo['id'].isin(port["id"])]

In [39]:
df_final_mesmo.to_csv("Similar_port_1.csv",index=False)

In [ ]:
# start_time = time.time()
# df_vazio=pd.DataFrame(columns=["id","similarity"])
# #itera mercado
# for i in range(len(df_dupl)):
#     doc_mercado = df_dupl["Docs"][i]
#     semelhanca_com_port=[]
#     #itera portfolio
#     for j in range(len(df_port_dupl)):
#         doc_portfolio = df_port_dupl["Docs"][j]
#         semelhanca_com_port.append(doc_mercado.similarity(doc_portfolio) * dici_port[df_port_dupl["0"][j]])
#     semelhanca_media = sum(semelhanca_com_port)/len(df_port)
#     df_vazio = df_vazio.append({'id':df_dupl['id'][i],'similarity':semelhanca_media},ignore_index=True)
#     print(i)
# tempo_total = (time.time() - start_time)
# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# tempo_total

In [ ]:
df_vazio.sort_values(by="similarity",ascending=False,ignore_index=True).head(530)

In [ ]:
df_vazio.sort_values("similarity",ascending=False).reset_index(drop=True)["id"][0]

In [ ]:
df[df["id"]=='44b1d031f0bff84d246181e97a7908a317cdfb8c0e096b937e08e5729c376d6b'].reset_index(drop=True)["0"][0]

In [ ]:
for token in df_dupl["Docs"][0]:
    print(token.text, token.lemma_, token.pos_, token.dep_, token.ent_type)

In [ ]:
# filename = 'modelo_mercado.sav'

In [ ]:
# doc_bin = DocBin(attrs=["POS", "TAG", "ENT_TYPE"], store_user_data=True)
# for doc in df22.Docs:
#     doc_bin.add(doc)
# bytes_data = doc_bin.to_bytes()
# pickle.dump(bytes_data, open( filename, "wb" ))

In [ ]:
# loaded_model = pickle.load(open(filename, 'rb'))
# doc_bin1 = DocBin(attrs=["POS", "TAG", "ENT_TYPE"]).from_bytes(bytes_data)
# docs = list(doc_bin.get_docs(nlp.vocab))

In [ ]:
# for token in docs[0]:
#     print(token.text, token.lemma_, token.pos_, token.dep_)

In [ ]:
# start_time = time.time()

# #itera mercado
# l_mercado=[]
# for i in range(10):
#     l_mercado.append(nlp(df["0"][i]))

# print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# #itera portfolio
# start_time = time.time()

# l_portfolio=[]
# for i in range(len(df_port_dupl)):
#     l_portfolio.append(nlp(df_port_dupl["0"][i]))
        
# print("--- %s seconds ---" % (time.time() - start_time))